In [1]:
import pandas as pd
import json
import statistics

df = pd.DataFrame()

for i in range(1, 24):
    tempdf = pd.DataFrame(columns=["year", "district", "rent"])
    tempdf["year"] = range(2012, 2025)
    with open("./data/rent_data/" + str(i) + ".json", "r") as f:
        text = json.loads(f.read())

    rows = text["Trends"]["TrendChart"]["Rows"]
    start = rows[:4]
    values = [i[1] for i in start]
    year_mean = [round(statistics.mean(values), 2)]
    del rows[:4]
    for j in range(0, len(rows), 12):
        year = rows[j:j+12]
        values = [val[1] for val in year]
        year_mean.append(round(statistics.mean(values), 2))
    
    tempdf["district"] = i
    tempdf["rent"] = year_mean
    
    df = pd.concat([df, tempdf], ignore_index=True)

tempdf = pd.DataFrame(columns=["year", "district", "rent"])
tempdf["year"] = range(2012, 2025)

overall = []
for i in range(2012, 2025):
    overall.append(round(statistics.mean(df["rent"][df["year"] == i]), 2))

tempdf["district"] = 0
tempdf["rent"] = overall
df = pd.concat([df, tempdf], ignore_index=True)
df

,year,district,rent
0,2012,1,19.63
1,2013,1,19.41
2,2014,1,19.45
3,2015,1,19.23
4,2016,1,19.18
...,...,...,...
307,2020,0,15.52
308,2021,0,15.79
309,2022,0,17.08
310,2023,0,17.42


In [2]:
df.to_csv("./data/average_rent_sqm.csv")

In [3]:
income_df = pd.read_csv("./data/vie-bez-biz-ecn-inc-sex-2002f.csv", sep=";", skiprows=1, usecols=[1,3,5,6,7])
rent_df = pd.read_csv("./data/average_rent_sqm.csv", index_col=0)

In [4]:
rent_df

,year,district,rent
0,2012,1,19.63
1,2013,1,19.41
2,2014,1,19.45
3,2015,1,19.23
4,2016,1,19.18
...,...,...,...
307,2020,0,15.52
308,2021,0,15.79
309,2022,0,17.08
310,2023,0,17.42


In [5]:
count = 0
for i in income_df["DISTRICT_CODE"].unique():
    income_df.loc[income_df["DISTRICT_CODE"] == i, "DISTRICT_CODE"] = count
    count += 1
income_df

,DISTRICT_CODE,REF_YEAR,INC_TOT_VALUE,INC_MAL_VALUE,INC_FEM_VALUE
0,0,2002,18.217,20.709,15.424
1,1,2002,25.463,31.961,18.536
2,2,2002,16.439,18.301,14.282
3,3,2002,18.701,21.444,15.804
4,4,2002,20.325,23.641,16.876
...,...,...,...,...,...
475,19,2021,29.903,34.540,25.235
476,20,2021,21.201,22.241,19.916
477,21,2021,24.381,26.587,21.926
478,22,2021,26.722,29.850,23.454


In [15]:
df = income_df.merge(rent_df, how="outer", left_on=["REF_YEAR", "DISTRICT_CODE"], right_on=["year", "district"])
df["district"] = df["district"].fillna(df["DISTRICT_CODE"])
df["year"] = df["year"].fillna(df["REF_YEAR"])
df = df.drop(["DISTRICT_CODE", "REF_YEAR"], axis = 1)

In [16]:
df

,INC_TOT_VALUE,INC_MAL_VALUE,INC_FEM_VALUE,year,district,rent
0,18.217,20.709,15.424,2002.0,0.0,NaN
1,25.463,31.961,18.536,2002.0,1.0,NaN
2,16.439,18.301,14.282,2002.0,2.0,NaN
3,18.701,21.444,15.804,2002.0,3.0,NaN
4,20.325,23.641,16.876,2002.0,4.0,NaN
...,...,...,...,...,...,...
547,NaN,NaN,NaN,2024.0,19.0,19.77
548,NaN,NaN,NaN,2024.0,20.0,18.25
549,NaN,NaN,NaN,2024.0,21.0,18.59
550,NaN,NaN,NaN,2024.0,22.0,22.72


In [17]:
mapping = {1: 901, 2: 902, 3: 903, 4: 904, 5: 905, 6: 906, 7: 907, 8: 908, 9: 909, 10: 910, 11: 911, 12: 912, 13: 913, 14: 914, 15: 915, 16: 916, 17: 917,
           18: 918, 19: 919, 20: 920, 21: 921, 22: 922, 23: 923}

df["district"] = list(map(mapping.get, df["district"]))
df["district"] = df["district"].fillna(0)
df

,INC_TOT_VALUE,INC_MAL_VALUE,INC_FEM_VALUE,year,district,rent
0,18.217,20.709,15.424,2002.0,0.0,NaN
1,25.463,31.961,18.536,2002.0,901.0,NaN
2,16.439,18.301,14.282,2002.0,902.0,NaN
3,18.701,21.444,15.804,2002.0,903.0,NaN
4,20.325,23.641,16.876,2002.0,904.0,NaN
...,...,...,...,...,...,...
547,NaN,NaN,NaN,2024.0,919.0,19.77
548,NaN,NaN,NaN,2024.0,920.0,18.25
549,NaN,NaN,NaN,2024.0,921.0,18.59
550,NaN,NaN,NaN,2024.0,922.0,22.72


In [18]:
df.to_csv("./data/rent_inc.csv")